In [1]:
import tensorflow as tf
import os
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg
import tensorflow.contrib.slim.nets as nets
from six.moves import cPickle as pickle

In [ ]:
from datasets import dataset_utils

url = "http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz"
checkpoints_dir = '/tmp/checkpoints'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

In [2]:
checkpoints_dir = '/tmp/checkpoints'
checkpt = os.path.join(checkpoints_dir, 'vgg_16.ckpt')

X = tf.placeholder(tf.float32, [None, 224, 224, 3])
with slim.arg_scope(vgg.vgg_arg_scope()):
    logits, end_points = vgg.vgg_16(X, is_training=False)

variables_to_restore = slim.get_variables_to_restore()
init_assign_op, init_feed_dict = slim.assign_from_checkpoint(checkpt, variables_to_restore)

In [3]:
saved_weights = {}

for i in init_feed_dict:
    print(i.name)
    saved_weights[str(i.name)] = init_feed_dict[i]

placeholder/vgg_16/conv3/conv3_1/biases:0
placeholder/vgg_16/conv2/conv2_1/biases:0
placeholder/vgg_16/conv1/conv1_2/weights:0
placeholder/vgg_16/conv3/conv3_2/weights:0
placeholder/vgg_16/fc8/weights:0
placeholder/vgg_16/fc7/biases:0
placeholder/vgg_16/fc7/weights:0
placeholder/vgg_16/conv5/conv5_2/biases:0
placeholder/vgg_16/fc8/biases:0
placeholder/vgg_16/fc6/weights:0
placeholder/vgg_16/conv4/conv4_1/weights:0
placeholder/vgg_16/conv5/conv5_1/weights:0
placeholder/vgg_16/conv1/conv1_2/biases:0
placeholder/vgg_16/conv4/conv4_2/weights:0
placeholder/vgg_16/conv1/conv1_1/weights:0
placeholder/vgg_16/conv5/conv5_2/weights:0
placeholder/vgg_16/conv1/conv1_1/biases:0
placeholder/vgg_16/conv4/conv4_3/weights:0
placeholder/vgg_16/conv2/conv2_1/weights:0
placeholder/vgg_16/conv3/conv3_3/weights:0
placeholder/vgg_16/fc6/biases:0
placeholder/vgg_16/conv4/conv4_2/biases:0
placeholder/vgg_16/conv3/conv3_1/weights:0
placeholder/vgg_16/conv2/conv2_2/biases:0
placeholder/vgg_16/conv5/conv5_3/weigh

In [4]:
print(saved_weights['placeholder/vgg_16/conv4/conv4_1/weights:0'].shape)
saved_weights['placeholder/vgg_16/conv4/conv4_1/weights:0']

(3, 3, 256, 512)


array([[[[ -1.24538150e-02,   8.40655994e-03,  -5.28845051e-03, ...,
           -2.04581907e-03,  -3.92359449e-03,   1.30683812e-03],
         [  1.15566896e-02,   2.46055960e-03,  -4.09082584e-02, ...,
           -4.92062652e-03,  -1.63423386e-03,   8.94230697e-03],
         [ -1.43469922e-04,  -5.65873180e-03,   1.84066985e-02, ...,
           -2.08840682e-03,   2.63321817e-06,  -2.50728261e-02],
         ..., 
         [  1.37888230e-02,  -6.58180192e-03,   9.65370331e-03, ...,
            7.17610447e-03,   3.45725450e-03,   5.33173978e-03],
         [  2.65833316e-03,   7.77704641e-04,   2.86239345e-04, ...,
           -5.40583069e-03,   8.93119723e-03,   1.40043022e-02],
         [ -8.53933115e-03,   1.43394794e-03,  -2.91879033e-03, ...,
            3.77660524e-03,   2.05454584e-02,  -1.35868033e-02]],

        [[ -1.20176468e-02,   1.51250837e-03,   3.43130133e-03, ...,
           -1.57830697e-02,  -2.39501009e-03,   2.93573504e-03],
         [  1.07167801e-02,  -1.72885903e-03,

In [5]:
pickle_file = 'vgg_weights.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'vgg_wights': saved_weights,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [12]:
def conv_layer(value):
    return tf.Variable(value)

In [13]:
graph = tf.Graph()

with graph.as_default():
    
  conv1_1_1 = conv_layer(saved_weights['placeholder/vgg_16/conv1/conv1_1/weights:0'])

In [18]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print(conv1_1_1.eval())

[[[[  4.80015397e-01  -1.72696680e-01   3.75577137e-02 ...,
     -1.27135560e-01  -5.02991639e-02   3.48965675e-02]
   [  5.50379455e-01   2.08774377e-02   9.88311544e-02 ...,
     -8.48205537e-02  -5.11389151e-02   3.74943428e-02]
   [  4.29470569e-01   1.17273867e-01   3.40129584e-02 ...,
     -1.32241577e-01  -5.33475243e-02   7.57738389e-03]]

  [[  4.08547401e-01  -1.70375049e-01  -4.96297423e-03 ...,
     -1.22360572e-01  -2.76450396e-01  -3.90796512e-02]
   [  4.40074533e-01   4.73412387e-02   5.13819456e-02 ...,
     -9.88498852e-02  -2.96195745e-01  -7.04357103e-02]
   [  3.73466998e-01   1.62062630e-01   1.70863140e-03 ...,
     -1.48207128e-01  -2.35300660e-01  -6.30356818e-02]]

  [[ -6.51455522e-02  -1.54351532e-01  -1.38038069e-01 ...,
     -1.29404560e-01  -4.62243795e-01  -3.23985279e-01]
   [ -8.13870355e-02   4.18543853e-02  -1.01763301e-01 ...,
     -9.43124294e-02  -5.05662560e-01  -3.83694321e-01]
   [ -6.13601133e-02   1.35693997e-01  -1.15694344e-01 ...,
     -1.